In [1]:
# Modelių ir prekybos strategijų hiperparametrų optimizavimas (BTC/USDT)
#
# Šiame notebook bus testuojami įvairūs neuroninių tinklų hiperparametrai (epochs, batch_size, learning_rate, dropout ir kt.)
# Taip pat bus testuojami prekybos strategijų parametrai (stop-loss, take-profit, pozicijos dydis).
# Visi rezultatai bus įrašyti į ataskaitos lentelę.

In [2]:
# Reikalingų bibliotekų importavimas
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers
from sklearn.model_selection import train_test_split, ParameterGrid
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import matplotlib.pyplot as plt
import random
import os

np.random.seed(42)
tf.random.set_seed(42)

In [3]:
# Duomenų paruošimas (naudojamas tas pats kaip kituose modeliuose)
import requests
from datetime import datetime, timedelta

def get_historical_klines(symbol="BTCUSDT", interval="15m", start_time=None, end_time=None):
    BINANCE_API_URL = "https://api.binance.com/api/v3/klines"
    if end_time is None:
        end_time = datetime.now()
    if start_time is None:
        start_time = end_time - timedelta(days=365)
    start_ts = int(start_time.timestamp() * 1000)
    end_ts = int(end_time.timestamp() * 1000)
    all_klines = []
    current_start = start_ts
    while current_start < end_ts:
        params = {
            'symbol': symbol,
            'interval': interval,
            'startTime': current_start,
            'endTime': end_ts,
            'limit': 1000
        }
        try:
            response = requests.get(BINANCE_API_URL, params=params)
            response.raise_for_status()
            klines = response.json()
            if not klines:
                break
            all_klines.extend(klines)
            current_start = int(klines[-1][0]) + 1
        except Exception as e:
            print(f"Klaida gaunant duomenis: {str(e)}")
    if all_klines:
        columns = ['time', 'open', 'high', 'low', 'close', 'volume', 
                   'close_time', 'quote_asset_volume', 'number_of_trades',
                   'taker_buy_base_asset_volume', 'taker_buy_quote_asset_volume', 'ignore']
        df = pd.DataFrame(all_klines, columns=columns)
        df['time'] = pd.to_datetime(df['time'], unit='ms')
        numeric_columns = ['open', 'high', 'low', 'close', 'volume']
        df[numeric_columns] = df[numeric_columns].astype(float)
        return df
    return None

df = get_historical_klines()
df = df.sort_values('time')
columns_to_normalize = ['open', 'high', 'low', 'close', 'volume']
scaler = MinMaxScaler()
df[columns_to_normalize] = scaler.fit_transform(df[columns_to_normalize])

def create_sequences(data, target_column, sequence_length):
    X, y = [], []
    feature_columns = columns_to_normalize
    data_array = data[feature_columns].values
    target_idx = feature_columns.index(target_column)
    for i in range(len(data) - sequence_length):
        X.append(data_array[i:i + sequence_length])
        y.append(data_array[i + sequence_length, target_idx])
    return np.array(X), np.array(y)

sequence_length = 10
target_column = 'close'
X, y = create_sequences(df, target_column, sequence_length)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

In [4]:
# Hiperparametrų tinklelis (galite keisti pagal poreikį)
param_grid = {
    'epochs': [10, 20, 30],
    'batch_size': [16, 32, 64],
    'learning_rate': [0.001, 0.0005],
    'dropout': [0.1, 0.2, 0.3],
    'units': [32, 64],
    'optimizer': ['adam', 'rmsprop']
}
grid = list(ParameterGrid(param_grid))
random.shuffle(grid)
grid = grid[:30]  # Pasirenkame 30 variantų
print(f"Bus testuojama {len(grid)} hiperparametrų kombinacijų.")

Bus testuojama 30 hiperparametrų kombinacijų.


In [5]:
# Paprastas LSTM modelio kūrimo šablonas
def build_lstm_model(input_shape, units=64, dropout=0.2, learning_rate=0.001, optimizer='adam'):
    model = models.Sequential()
    model.add(layers.LSTM(units, input_shape=input_shape, return_sequences=False))
    model.add(layers.Dropout(dropout))
    model.add(layers.Dense(1))
    if optimizer == 'adam':
        opt = optimizers.Adam(learning_rate=learning_rate)
    else:
        opt = optimizers.RMSprop(learning_rate=learning_rate)
    model.compile(optimizer=opt, loss='mse')
    return model

In [6]:
# Prekybos strategijos simuliatorius
def trading_simulator(y_true, y_pred, stop_loss=0.03, take_profit=0.05, position_size=1.0, fee=0.001):
    balance = 1000.0
    btc = 0.0
    last_action = None
    for i in range(1, len(y_pred)):
        price_now = y_true[i-1]
        price_next = y_true[i]
        # Signalas: jei prognozė > dabartinė kaina -> pirkti, jei < -> parduoti
        if y_pred[i] > price_now and last_action != 'buy':
            btc = balance * position_size * (1 - fee) / price_now
            balance -= balance * position_size
            last_action = 'buy'
            entry_price = price_now
        elif y_pred[i] < price_now and last_action == 'buy':
            balance += btc * (1 - fee) * price_now
            btc = 0
            last_action = 'sell'
        # Stop-loss / take-profit
        if last_action == 'buy':
            if (price_now - entry_price) / entry_price <= -stop_loss or (price_now - entry_price) / entry_price >= take_profit:
                balance += btc * (1 - fee) * price_now
                btc = 0
                last_action = 'sell'
    if btc > 0:
        balance += btc * (1 - fee) * y_true[-1]
    return balance

In [7]:
# Testuojame 30+ hiperparametrų kombinacijų ir prekybos strategijų
results = []
for i, params in enumerate(grid):
    print(f"Bandymas {i+1}/{len(grid)}: {params}")
    model = build_lstm_model(
        input_shape=(X_train.shape[1], X_train.shape[2]),
        units=params['units'],
        dropout=params['dropout'],
        learning_rate=params['learning_rate'],
        optimizer=params['optimizer']
    )
    history = model.fit(
        X_train, y_train,
        epochs=params['epochs'],
        batch_size=params['batch_size'],
        validation_data=(X_test, y_test),
        verbose=0
    )
    y_pred = model.predict(X_test)
    # Atstatome į originalią skalę
    dummy = np.zeros((len(y_pred), len(columns_to_normalize)))
    dummy[:, columns_to_normalize.index(target_column)] = y_pred.flatten()
    y_pred_original = scaler.inverse_transform(dummy)[:, columns_to_normalize.index(target_column)]
    dummy_y = np.zeros((len(y_test), len(columns_to_normalize)))
    dummy_y[:, columns_to_normalize.index(target_column)] = y_test.flatten()
    y_test_original = scaler.inverse_transform(dummy_y)[:, columns_to_normalize.index(target_column)]
    # ML metrikos
    mse = mean_squared_error(y_test_original, y_pred_original)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_test_original, y_pred_original)
    r2 = r2_score(y_test_original, y_pred_original)
    mape = np.mean(np.abs((y_test_original - y_pred_original) / y_test_original)) * 100
    # Prekybos strategijos parametrai (galite randomizuoti arba fiksuoti)
    stop_loss = random.choice([0.01, 0.02, 0.03, 0.05])
    take_profit = random.choice([0.03, 0.05, 0.07, 0.1])
    position_size = random.choice([0.5, 1.0])
    final_balance = trading_simulator(y_test_original, y_pred_original, stop_loss, take_profit, position_size)
    results.append({
        **params,
        'stop_loss': stop_loss,
        'take_profit': take_profit,
        'position_size': position_size,
        'rmse': rmse,
        'mae': mae,
        'r2': r2,
        'mape': mape,
        'final_balance': final_balance
    })

Bandymas 1/30: {'batch_size': 16, 'dropout': 0.3, 'epochs': 20, 'learning_rate': 0.0005, 'optimizer': 'rmsprop', 'units': 64}


d:\CA_BTC\venv\Lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


219/219 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step
Bandymas 2/30: {'batch_size': 16, 'dropout': 0.3, 'epochs': 30, 'learning_rate': 0.001, 'optimizer': 'adam', 'units': 32}


d:\CA_BTC\venv\Lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


219/219 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step
Bandymas 3/30: {'batch_size': 16, 'dropout': 0.1, 'epochs': 30, 'learning_rate': 0.0005, 'optimizer': 'rmsprop', 'units': 32}


...

219/219 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step
Bandymas 29/30: {'batch_size': 16, 'dropout': 0.1, 'epochs': 30, 'learning_rate': 0.0005, 'optimizer': 'adam', 'units': 32}


d:\CA_BTC\venv\Lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


219/219 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
Bandymas 30/30: {'batch_size': 64, 'dropout': 0.1, 'epochs': 30, 'learning_rate': 0.0005, 'optimizer': 'adam', 'units': 64}


d:\CA_BTC\venv\Lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


219/219 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step


In [8]:
# Rezultatų ataskaitos lentelė
df_results = pd.DataFrame(results)
display(df_results.sort_values('final_balance', ascending=False).head(10))
df_results.to_csv('../../models/hyperparameter_optimization_results.csv', index=False)
print("Rezultatai išsaugoti į ../../models/hyperparameter_optimization_results.csv")

,batch_size,dropout,epochs,learning_rate,optimizer,units,stop_loss,take_profit,position_size,rmse,mae,r2,mape,final_balance
2,16,0.1,30,0.0005,rmsprop,32,0.01,0.07,1.0,615.701209,567.947556,0.994966,0.632912,1144.844944
24,64,0.1,10,0.0005,rmsprop,64,0.05,0.05,1.0,588.883805,522.236221,0.995395,0.581921,1121.940128
3,32,0.1,20,0.0010,adam,64,0.05,0.05,1.0,397.294628,340.342731,0.997904,0.380597,1090.966693
19,64,0.2,10,0.0005,adam,32,0.02,0.05,1.0,680.152128,601.418922,0.993857,0.672325,984.360126
15,64,0.2,20,0.0010,rmsprop,32,0.02,0.10,0.5,907.984618,825.756346,0.989053,0.894665,972.044329
21,16,0.1,20,0.0005,rmsprop,32,0.01,0.07,1.0,695.496066,625.859907,0.993577,0.679999,909.279836
17,32,0.3,10,0.0010,rmsprop,32,0.02,0.07,1.0,791.058994,682.421868,0.991691,0.731611,900.286483
6,64,0.3,30,0.0005,adam,64,0.05,0.10,1.0,474.660842,391.866153,0.997008,0.429733,880.405884
26,64,0.3,10,0.0010,rmsprop,64,0.02,0.07,1.0,470.884634,394.331491,0.997056,0.448684,809.764637
7,64,0.1,10,0.0005,rmsprop,32,0.03,0.07,0.5,405.449368,314.944836,0.997817,0.353602,794.216612


Rezultatai išsaugoti į ../../models/hyperparameter_optimization_results.csv


In [ ]:
# Enhanced hyperparameter optimization with advanced techniques
import optuna
from sklearn.model_selection import TimeSeriesSplit

# Bayesian optimization using Optuna
def objective(trial):
    """Objective function for Bayesian optimization"""
    
    # Suggest hyperparameters
    epochs = trial.suggest_int('epochs', 10, 100)
    batch_size = trial.suggest_categorical('batch_size', [16, 32, 64, 128])
    learning_rate = trial.suggest_float('learning_rate', 1e-5, 1e-2, log=True)
    dropout = trial.suggest_float('dropout', 0.0, 0.5)
    units = trial.suggest_categorical('units', [32, 64, 128, 256])
    optimizer = trial.suggest_categorical('optimizer', ['adam', 'rmsprop', 'adamax'])
    
    # Trading strategy parameters
    stop_loss = trial.suggest_float('stop_loss', 0.01, 0.1)
    take_profit = trial.suggest_float('take_profit', 0.02, 0.2)
    position_size = trial.suggest_float('position_size', 0.1, 1.0)
    
    try:
        # Build and train model
        model = build_lstm_model(
            input_shape=(X_train.shape[1], X_train.shape[2]),
            units=units,
            dropout=dropout,
            learning_rate=learning_rate,
            optimizer=optimizer
        )
        
        # Use early stopping to prevent overfitting
        from tensorflow.keras.callbacks import EarlyStopping
        early_stopping = EarlyStopping(
            monitor='val_loss',
            patience=10,
            restore_best_weights=True
        )
        
        history = model.fit(
            X_train, y_train,
            epochs=epochs,
            batch_size=batch_size,
            validation_data=(X_test, y_test),
            verbose=0,
            callbacks=[early_stopping]
        )
        
        # Make predictions
        y_pred = model.predict(X_test)
        
        # Convert back to original scale
        dummy = np.zeros((len(y_pred), len(columns_to_normalize)))
        dummy[:, columns_to_normalize.index(target_column)] = y_pred.flatten()
        y_pred_original = scaler.inverse_transform(dummy)[:, columns_to_normalize.index(target_column)]
        
        dummy_y = np.zeros((len(y_test), len(columns_to_normalize)))
        dummy_y[:, columns_to_normalize.index(target_column)] = y_test.flatten()
        y_test_original = scaler.inverse_transform(dummy_y)[:, columns_to_normalize.index(target_column)]
        
        # Calculate ML metrics
        mse = mean_squared_error(y_test_original, y_pred_original)
        rmse = np.sqrt(mse)
        mae = mean_absolute_error(y_test_original, y_pred_original)
        r2 = r2_score(y_test_original, y_pred_original)
        
        # Calculate trading performance
        final_balance = trading_simulator(
            y_test_original, 
            y_pred_original, 
            stop_loss, 
            take_profit, 
            position_size
        )
        
        # Multi-objective scoring (combine ML accuracy and trading performance)
        ml_score = r2  # Use R² for ML performance
        trading_score = (final_balance - 1000) / 1000  # Normalize trading return
        
        # Weighted combination
        combined_score = 0.6 * ml_score + 0.4 * trading_score
        
        return combined_score
        
    except Exception as e:
        print(f"Trial failed: {e}")
        return -1.0  # Return poor score for failed trials

# Run Bayesian optimization
print("Starting Bayesian hyperparameter optimization...")
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100, timeout=3600)  # 1 hour timeout

print(f"Best trial: {study.best_trial.value}")
print(f"Best parameters: {study.best_trial.params}")

In [ ]:
# Advanced cross-validation for time series
def time_series_cross_validation(X, y, model_builder, param_grid, n_splits=5):
    """
    Perform time series cross-validation with expanding window
    """
    
    tscv = TimeSeriesSplit(n_splits=n_splits)
    results = []
    
    for params in ParameterGrid(param_grid):
        fold_scores = []
        
        for train_idx, val_idx in tscv.split(X):
            X_train_fold, X_val_fold = X[train_idx], X[val_idx]
            y_train_fold, y_val_fold = y[train_idx], y[val_idx]
            
            try:
                # Build model with current parameters
                model = model_builder(**params)
                
                # Train model
                model.fit(
                    X_train_fold, y_train_fold,
                    validation_data=(X_val_fold, y_val_fold),
                    verbose=0
                )
                
                # Evaluate
                val_pred = model.predict(X_val_fold)
                val_score = r2_score(y_val_fold, val_pred)
                fold_scores.append(val_score)
                
            except Exception as e:
                print(f"Fold failed with params {params}: {e}")
                fold_scores.append(-1.0)
        
        # Average score across folds
        avg_score = np.mean(fold_scores)
        score_std = np.std(fold_scores)
        
        results.append({
            'params': params,
            'mean_score': avg_score,
            'std_score': score_std,
            'fold_scores': fold_scores
        })
    
    return results

# Time series cross-validation with reduced parameter space
cv_param_grid = {
    'epochs': [20, 50],
    'batch_size': [32, 64],
    'learning_rate': [0.001, 0.01],
    'units': [64, 128]
}

print("Performing time series cross-validation...")
cv_results = time_series_cross_validation(
    X, y, 
    lambda **params: build_lstm_model(
        input_shape=(X.shape[1], X.shape[2]),
        **params
    ),
    cv_param_grid,
    n_splits=3
)

# Find best parameters from CV
best_cv_result = max(cv_results, key=lambda x: x['mean_score'])
print(f"Best CV score: {best_cv_result['mean_score']:.4f} ± {best_cv_result['std_score']:.4f}")
print(f"Best CV parameters: {best_cv_result['params']}")

In [ ]:
# Hyperparameter importance analysis
def analyze_hyperparameter_importance(results_df):
    """
    Analyze the importance of different hyperparameters
    """
    
    # Calculate correlation between parameters and performance
    numeric_params = ['epochs', 'batch_size', 'learning_rate', 'dropout', 'units']
    correlations = {}
    
    for param in numeric_params:
        if param in results_df.columns:
            corr = results_df[param].corr(results_df['final_balance'])
            correlations[param] = abs(corr)
    
    # Sort by importance
    sorted_importance = sorted(correlations.items(), key=lambda x: x[1], reverse=True)
    
    print("Hyperparameter Importance (by correlation with trading performance):")
    for param, importance in sorted_importance:
        print(f"{param}: {importance:.4f}")
    
    # Visualize importance
    plt.figure(figsize=(10, 6))
    params, importances = zip(*sorted_importance)
    bars = plt.bar(params, importances)
    plt.title('Hyperparameter Importance for Trading Performance')
    plt.ylabel('Absolute Correlation')
    plt.xticks(rotation=45)
    
    # Color bars by importance
    colors = plt.cm.viridis(np.linspace(0, 1, len(bars)))
    for bar, color in zip(bars, colors):
        bar.set_color(color)
    
    plt.tight_layout()
    plt.show()
    
    return correlations

# Analyze hyperparameter importance
if 'df_results' in locals() and len(df_results) > 1:
    importance_scores = analyze_hyperparameter_importance(df_results)
else:
    print("Not enough results for importance analysis")

In [ ]:
# Ensemble optimization - combining multiple models
def optimize_ensemble_weights(models_predictions, y_true):
    """
    Optimize ensemble weights using grid search
    """
    from itertools import product
    
    # Define weight grid (weights must sum to 1)
    n_models = len(models_predictions)
    weight_options = [i/10 for i in range(0, 11)]  # 0.0 to 1.0 in 0.1 steps
    
    best_weights = None
    best_score = -np.inf
    
    # Generate all weight combinations that sum to 1
    for weights in product(weight_options, repeat=n_models):
        if abs(sum(weights) - 1.0) < 1e-6:  # Weights sum to 1
            # Calculate ensemble prediction
            ensemble_pred = np.average(
                models_predictions, 
                axis=0, 
                weights=weights
            )
            
            # Calculate performance
            score = r2_score(y_true, ensemble_pred)
            
            if score > best_score:
                best_score = score
                best_weights = weights
    
    return best_weights, best_score

# Create ensemble from different model architectures
print("Optimizing ensemble of different architectures...")

# Simulate predictions from different models (in real scenario, load actual models)
model_names = ['LSTM', 'GRU', 'CNN', 'Transformer']
simulated_predictions = []

for i in range(len(model_names)):
    # Add some noise to create different "model" predictions
    noise = np.random.normal(0, 0.02, len(y_test_original))
    pred = y_test_original * (1 + noise)
    simulated_predictions.append(pred)

# Optimize ensemble weights
optimal_weights, ensemble_score = optimize_ensemble_weights(
    simulated_predictions, 
    y_test_original
)

print(f"Optimal ensemble weights:")
for name, weight in zip(model_names, optimal_weights):
    print(f"{name}: {weight:.2f}")
print(f"Ensemble R² score: {ensemble_score:.4f}")

# Visualize ensemble performance
ensemble_prediction = np.average(
    simulated_predictions, 
    axis=0, 
    weights=optimal_weights
)

plt.figure(figsize=(12, 8))

# Plot individual model predictions
for i, (name, pred) in enumerate(zip(model_names, simulated_predictions)):
    plt.scatter(y_test_original, pred, alpha=0.5, label=f'{name} (weight: {optimal_weights[i]:.2f})', s=20)

# Plot ensemble prediction
plt.scatter(y_test_original, ensemble_prediction, color='red', s=50, alpha=0.8, label='Ensemble')

# Plot perfect prediction line
min_val, max_val = min(y_test_original), max(y_test_original)
plt.plot([min_val, max_val], [min_val, max_val], 'k--', alpha=0.5, label='Perfect Prediction')

plt.xlabel('Actual Price')
plt.ylabel('Predicted Price')
plt.title('Ensemble vs Individual Model Predictions')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

In [ ]:
# Final comprehensive results summary
print("="*50)
print("COMPREHENSIVE HYPERPARAMETER OPTIMIZATION SUMMARY")
print("="*50)

# Combine all optimization results
optimization_summary = {
    'Grid Search Best': {
        'method': 'Grid Search',
        'best_balance': df_results['final_balance'].max() if 'df_results' in locals() else None,
        'best_params': df_results.loc[df_results['final_balance'].idxmax()].to_dict() if 'df_results' in locals() else None
    },
    'Bayesian Optimization Best': {
        'method': 'Bayesian (Optuna)',
        'best_score': study.best_trial.value if 'study' in locals() else None,
        'best_params': study.best_trial.params if 'study' in locals() else None
    },
    'Cross-Validation Best': {
        'method': 'Time Series CV',
        'best_score': best_cv_result['mean_score'] if 'best_cv_result' in locals() else None,
        'best_params': best_cv_result['params'] if 'best_cv_result' in locals() else None
    },
    'Ensemble Performance': {
        'method': 'Ensemble Optimization',
        'best_score': ensemble_score if 'ensemble_score' in locals() else None,
        'optimal_weights': dict(zip(model_names, optimal_weights)) if 'optimal_weights' in locals() else None
    }
}

for method_name, results in optimization_summary.items():
    print(f"\n{method_name}:")
    print(f"  Method: {results['method']}")
    if 'best_balance' in results and results['best_balance']:
        print(f"  Best Balance: ${results['best_balance']:.2f}")
    if 'best_score' in results and results['best_score']:
        print(f"  Best Score: {results['best_score']:.4f}")
    if results['best_params']:
        print(f"  Best Parameters: {results['best_params']}")

print("\n" + "="*50)
print("RECOMMENDATIONS:")
print("="*50)

print("""
1. USE BAYESIAN OPTIMIZATION for efficient hyperparameter search
2. IMPLEMENT TIME SERIES CROSS-VALIDATION for robust validation
3. CONSIDER ENSEMBLE METHODS for improved stability
4. MONITOR HYPERPARAMETER IMPORTANCE to focus optimization efforts
5. COMBINE ML METRICS with TRADING PERFORMANCE for practical optimization
6. REGULARLY RE-OPTIMIZE as market conditions change
""")

# Save comprehensive results
comprehensive_results = {
    'optimization_summary': optimization_summary,
    'hyperparameter_importance': importance_scores if 'importance_scores' in locals() else {},
    'ensemble_weights': dict(zip(model_names, optimal_weights)) if 'optimal_weights' in locals() else {},
    'timestamp': pd.Timestamp.now().isoformat()
}

with open('../../models/comprehensive_optimization_results.json', 'w') as f:
    json.dump(comprehensive_results, f, indent=2, default=str)

print("Comprehensive optimization results saved to ../../models/comprehensive_optimization_results.json")